In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report as cr
import pickle

warnings.simplefilter(action = 'ignore', category = FutureWarning)
k_folds = KFold(n_splits = 10)

In [2]:
dataset = pd.read_csv("Hotel_dataset.csv")
dataset.drop(dataset.tail(15491).index, inplace = True)
reviews = [w for w in dataset["Review"]]

In [3]:
dataset.loc[dataset.Rating == 1, 'Rating'] = 0
dataset.loc[dataset.Rating == 2, 'Rating'] = 0
dataset.loc[dataset.Rating == 3, 'Rating'] = 0
dataset.loc[dataset.Rating == 4, 'Rating'] = 1
dataset.loc[dataset.Rating == 5, 'Rating'] = 1

In [4]:
pickle_in = open("elmo_vectors.pickle", "rb")
word_vectors = pickle.load(pickle_in)

In [16]:
def getReviewVector(review):
    return np.mean([word for word in review], axis = 0)

In [17]:
x = []
for i in range(len(word_vectors)):
    x.append(getReviewVector(word_vectors[i]))

In [18]:
y = pd.get_dummies(dataset["Rating"])
y = y.iloc[:,1].values

In [19]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [20]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [21]:
score = accuracy_score(y_test, y_pred)
print(score)
print(cr(y_test, y_pred))

0.838
              precision    recall  f1-score   support

           0       0.85      0.57      0.68       305
           1       0.84      0.95      0.89       695

    accuracy                           0.84      1000
   macro avg       0.84      0.76      0.79      1000
weighted avg       0.84      0.84      0.83      1000



In [22]:
# 10-fold cross-validation
scores = cross_val_score(rfc, x, y, cv = k_folds)

In [23]:
# 10-fold cross-validation results
print("CV scores: ", scores)
print("Avg CV score: ", scores.mean())

CV scores:  [0.8   0.866 0.846 0.874 0.836 0.788 0.782 0.834 0.87  0.896]
Avg CV score:  0.8392000000000002
